<a href="https://colab.research.google.com/github/YasminGarcia1210/MAESTRIA_DATOS/blob/main/Tarea_1_B_Profiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import os
os.environ["SPARK_VERSION"] = '3.1'

In [3]:
import os
# indicate your Spark version, here we use Spark 3.5 with pydeequ 1.4.0
os.environ["SPARK_VERSION"] = '3.5'

!pip install pydeequ
!pip install sagemaker-pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 MB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 14.6 MB/s eta 0:00:00
  Created wheel for sagemaker-pyspark: filename=sagemaker_pyspark-1.4.5-py3-none-any.whl size=181610593 sha256=3d68f1d62645da17f3a2b08a05682322c9e2c3740a09665f9ce303194cab73b0
  Stored in directory: /root/.cache/pip/wheels/ea/cb/32/140bffbc4ad8465e99b41cd848723a287b07650044483c62fa
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764003 sha256=a93d6b9833068c12d9ebe07290703c623bc1b6ac5fee5dda2f57989db2cbf1cb
  Stored in directory: /root/.cache/pip/wheels/81/9c/6c/d5200fcf351ffa39cbe09911e99703283624cd037df58070d9
Successfully built sagemaker-pyspark pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
  

In [47]:
from pydeequ.anomaly_detection import *
from pydeequ.analyzers import *

In [48]:
import pydeequ

import sagemaker_pyspark
from pyspark.sql import SparkSession, Row

classpath = ":".join(sagemaker_pyspark.classpath_jars()) # aws-specific jars

spark = (SparkSession
    .builder
    .config("spark.driver.extraClassPath", classpath)
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())


In [49]:
df = spark.read.csv("test.csv",header=True, inferSchema=True)
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- battery_power: integer (nullable = true)
 |-- blue: integer (nullable = true)
 |-- clock_speed: double (nullable = true)
 |-- dual_sim: integer (nullable = true)
 |-- fc: integer (nullable = true)
 |-- four_g: integer (nullable = true)
 |-- int_memory: integer (nullable = true)
 |-- m_dep: double (nullable = true)
 |-- mobile_wt: integer (nullable = true)
 |-- n_cores: integer (nullable = true)
 |-- pc: integer (nullable = true)
 |-- px_height: integer (nullable = true)
 |-- px_width: integer (nullable = true)
 |-- ram: integer (nullable = true)
 |-- sc_h: integer (nullable = true)
 |-- sc_w: integer (nullable = true)
 |-- talk_time: integer (nullable = true)
 |-- three_g: integer (nullable = true)
 |-- touch_screen: integer (nullable = true)
 |-- wifi: integer (nullable = true)



In [38]:
!sudo update-alternatives --config java

There is only one alternative in link group java (providing /usr/bin/java): /usr/lib/jvm/java-11-openjdk-amd64/bin/java
Nothing to configure.


In [41]:
import pydeequ
from pyspark.sql import SparkSession
spark = (SparkSession
         .builder
         .config("spark.jars.packages", pydeequ.deequ_maven_coord)
         .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
         .getOrCreate())

In [50]:
result = ColumnProfilerRunner(spark).onData(df).run()

Py4JJavaError: An error occurred while calling o161.run.
: com.amazon.deequ.analyzers.runners.MetricCalculationRuntimeException: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 20.0 failed 1 times, most recent failure: Lost task 0.0 in stage 20.0 (TID 20) (c77c50d9333d executor driver): java.lang.NoClassDefFoundError: org/apache/spark/sql/catalyst/types/DataTypeUtils$
	at org.apache.spark.sql.catalyst.expressions.aggregate.StatefulHyperloglogPlus.aggBufferSchema(StatefulHyperloglogPlus.scala:63)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.$anonfun$initializeAggregateFunctions$1(AggregationIterator.scala:110)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.$anonfun$initializeAggregateFunctions$1$adapted(AggregationIterator.scala:81)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.initializeAggregateFunctions(AggregationIterator.scala:81)
	

In [43]:
result = ColumnProfilerRunner(spark).onData(df).run()
for col, profile in result.profiles.items():
    print(f'Column \'{col}\'')
    print('\t',f'completeness: {profile.completeness}')
    print('\t',f'approximate number of distinct values: {profile.approximateNumDistinctValues}')
    print('\t',f'datatype: {profile.dataType}')

Py4JJavaError: An error occurred while calling o146.run.
: com.amazon.deequ.analyzers.runners.MetricCalculationRuntimeException: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 17.0 failed 1 times, most recent failure: Lost task 0.0 in stage 17.0 (TID 17) (c77c50d9333d executor driver): java.lang.NoClassDefFoundError: org/apache/spark/sql/catalyst/types/DataTypeUtils$
	at org.apache.spark.sql.catalyst.expressions.aggregate.StatefulHyperloglogPlus.aggBufferSchema(StatefulHyperloglogPlus.scala:63)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.$anonfun$initializeAggregateFunctions$1(AggregationIterator.scala:110)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.$anonfun$initializeAggregateFunctions$1$adapted(AggregationIterator.scala:81)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.initializeAggregateFunctions(AggregationIterator.scala:81)
	

In [51]:
for col, profile in result.profiles.items():
    print(f'Column \'{col}\'')
    print('\t',f'completeness: {profile.completeness}')
    print('\t',f'approximate number of distinct values: {profile.approximateNumDistinctValues}')
    print('\t',f'datatype: {profile.dataType}')

AttributeError: 'ColumnProfilerRunBuilder' object has no attribute 'profiles'